In [1]:
# MLP - with using paper Bengio et al.2003

In [2]:
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt # for making figures
%matplotlib inline

In [3]:
# read in all the words
words = open('cleaned_names.txt', 'r').read().splitlines()
words[:8]

['aaban', 'aabid', 'aabidah', 'aabir', 'aabriella', 'aada', 'aadam', 'aadarsh']

In [4]:
len(words)

29681

In [5]:
# build the vocabulary of characters and mappings to/from integers
chars = sorted(set(''.join(words)))
stoi = {s:i+1 for i,s in enumerate(chars)}
stoi['.'] = 0
itos = {i:s for s,i in stoi.items()}
print(itos)

{1: 'a', 2: 'b', 3: 'c', 4: 'd', 5: 'e', 6: 'f', 7: 'g', 8: 'h', 9: 'i', 10: 'j', 11: 'k', 12: 'l', 13: 'm', 14: 'n', 15: 'o', 16: 'p', 17: 'q', 18: 'r', 19: 's', 20: 't', 21: 'u', 22: 'v', 23: 'w', 24: 'x', 25: 'y', 26: 'z', 0: '.'}


In [6]:
# build the dataset

block_size = 3 # context length: how many characters to use to predict the next one?
X, Y = [], []
for w in words: # [:5]

  # print(w)
  context = [0] * block_size
  for ch in w + '.':
    ix = stoi[ch]
    X.append(context)
    Y.append(ix)
    # print(''.join([itos[i] for i in context]), '---->', itos[ix])
    context = context[1:] + [ix] # crop and append

X = torch.tensor(X)
Y = torch.tensor(Y)

In [7]:
X.shape, X.dtype, Y.shape, Y.dtype 

(torch.Size([212725, 3]), torch.int64, torch.Size([212725]), torch.int64)

In [8]:
X

tensor([[ 0,  0,  0],
        [ 0,  0,  1],
        [ 0,  1,  1],
        ...,
        [26, 26, 25],
        [26, 25, 26],
        [25, 26, 24]])

In [9]:
Y # labels

tensor([ 1,  1,  2,  ..., 26, 24,  0])

In [10]:
C = torch.randn(27, 2) # each 27 characters is represented by a 2D embedding

In [11]:
C

tensor([[ 0.0476,  1.0978],
        [-0.2539,  0.2764],
        [-0.0310, -0.7259],
        [ 1.1202,  0.1660],
        [-0.2143, -0.6151],
        [ 0.0047,  0.4296],
        [-1.2551, -0.2133],
        [-0.2876, -0.5761],
        [-0.7576,  1.3757],
        [ 0.0230, -0.8956],
        [-1.1389, -0.9445],
        [ 0.3078,  0.9970],
        [-0.4728, -1.2626],
        [ 0.1584,  0.4598],
        [ 1.2565, -0.7817],
        [ 0.3753,  0.3339],
        [ 0.3999, -0.0972],
        [-0.9135,  0.8395],
        [-1.3970, -0.5593],
        [ 0.0756, -0.2986],
        [-0.6856, -1.1085],
        [ 0.5505,  0.7247],
        [-0.9849,  0.7696],
        [-0.6548, -0.8361],
        [ 0.0271, -2.6902],
        [ 1.3252, -1.6531],
        [-0.7951,  0.0327]])

In [12]:
C[5]

tensor([0.0047, 0.4296])

In [13]:
F.one_hot(torch.tensor(5), num_classes=27).float() @ C # one hot vector

tensor([0.0047, 0.4296])

In [14]:
C[[5,6,7]] # indexing with list

tensor([[ 0.0047,  0.4296],
        [-1.2551, -0.2133],
        [-0.2876, -0.5761]])

In [15]:
C[torch.tensor([5,6,7])] # indexing with tensor

tensor([[ 0.0047,  0.4296],
        [-1.2551, -0.2133],
        [-0.2876, -0.5761]])

In [16]:
C[X].shape

torch.Size([212725, 3, 2])

In [17]:
X[13,2] # 13th example, 2nd character

tensor(1)

In [18]:
C[X[13,2]] # embedding of the 13th example, 2nd character

tensor([-0.2539,  0.2764])

In [19]:
C[1]

tensor([-0.2539,  0.2764])

In [20]:
emb = C[X]
emb.shape

torch.Size([212725, 3, 2])

In [21]:
# Constructing the hidden layer
W1 = torch.randn((6, 100))
b1 = torch.randn(100)

In [22]:
h = emb.view(-1, 6) @ W1 + b1

In [23]:
h

tensor([[ -3.2715,  -1.8944,   0.2583,  ...,   4.2108,   1.6700,   1.5456],
        [ -3.5034,  -0.4649,   1.5156,  ...,   1.5958,   2.1645,  -0.1849],
        [ -3.5268,  -0.0525,   0.8714,  ...,   1.0409,   1.9500,   0.2217],
        ...,
        [ -4.2173,   3.1079,  -1.9717,  ...,  -2.7761,  -0.3052,  -2.8323],
        [ -3.1089,   0.1678,   0.5640,  ...,  -0.3921,  -7.6828,   2.1024],
        [  0.8888,  -0.0373,   0.6550,  ..., -12.5619,   0.1803,  -4.3022]])

In [24]:
# torch.cat([emb[:, 0, :], emb[:, 1, :], emb[:, 2, :]], 1).shape # first, second, third characters of the first example concatenated

In [25]:
# torch.cat(torch.unbind(emb, 1), 1).shape # split the tensor along the second dimension

In [26]:
a = torch.arange(18)
a

tensor([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17])

In [27]:
a.shape

torch.Size([18])

In [28]:
a.view(2, 9) # view as 2x9 matrix

tensor([[ 0,  1,  2,  3,  4,  5,  6,  7,  8],
        [ 9, 10, 11, 12, 13, 14, 15, 16, 17]])

In [29]:
a.view(9, 2)

tensor([[ 0,  1],
        [ 2,  3],
        [ 4,  5],
        [ 6,  7],
        [ 8,  9],
        [10, 11],
        [12, 13],
        [14, 15],
        [16, 17]])

In [30]:
a.view(3, 3, 2)

tensor([[[ 0,  1],
         [ 2,  3],
         [ 4,  5]],

        [[ 6,  7],
         [ 8,  9],
         [10, 11]],

        [[12, 13],
         [14, 15],
         [16, 17]]])

In [31]:
a.storage() # memory address

C:\Users\DC\AppData\Local\Temp\ipykernel_15412\3903475333.py:1: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  a.storage() # memory address


 0
 1
 2
 3
 4
 5
 6
 7
 8
 9
 10
 11
 12
 13
 14
 15
 16
 17
[torch.storage.TypedStorage(dtype=torch.int64, device=cpu) of size 18]

In [32]:
emb.shape

torch.Size([212725, 3, 2])

In [33]:
# emb.view(32, 6) == torch.cat(torch.unbind(emb, 1), 1)

In [34]:
h = torch.tanh(emb.view(-1, 6) @ W1 + b1)

In [35]:
h

tensor([[-0.9971, -0.9558,  0.2527,  ...,  0.9996,  0.9316,  0.9131],
        [-0.9982, -0.4341,  0.9079,  ...,  0.9210,  0.9740, -0.1828],
        [-0.9983, -0.0525,  0.7021,  ...,  0.7782,  0.9603,  0.2181],
        ...,
        [-0.9996,  0.9960, -0.9620,  ..., -0.9923, -0.2960, -0.9931],
        [-0.9960,  0.1663,  0.5110,  ..., -0.3732, -1.0000,  0.9706],
        [ 0.7108, -0.0373,  0.5750,  ..., -1.0000,  0.1783, -0.9996]])

In [36]:
h.shape

torch.Size([212725, 100])

In [37]:
(emb.view(-1, 6) @ W1).shape

torch.Size([212725, 100])

In [38]:
b1.shape

torch.Size([100])

In [39]:
# 32, 100
# broadcasting to 100
# 1 , 100

In [40]:
# Creating the final layer (output layer)
W2 = torch.randn(100, 27)
b2 = torch.randn(27)

In [41]:
logits = h @ W2 + b2

In [42]:
logits.shape

torch.Size([212725, 27])

In [43]:
counts = logits.exp()

In [44]:
prob = counts / counts.sum(-1, keepdim=True)

In [45]:
prob.shape

torch.Size([212725, 27])

In [46]:
prob

tensor([[8.2905e-03, 1.6034e-03, 4.0720e-10,  ..., 1.4775e-09, 7.1503e-11,
         5.2379e-03],
        [1.5175e-03, 3.7170e-03, 2.6218e-11,  ..., 2.1562e-08, 5.5273e-09,
         1.2925e-02],
        [7.5443e-06, 5.7129e-04, 4.7702e-09,  ..., 3.6178e-10, 7.9139e-08,
         3.8941e-03],
        ...,
        [1.0774e-06, 5.2451e-05, 4.1313e-08,  ..., 7.8999e-04, 3.0146e-02,
         3.9948e-14],
        [1.4006e-05, 1.4336e-08, 3.5731e-06,  ..., 2.8481e-07, 3.2863e-07,
         1.6426e-03],
        [2.1113e-07, 5.7196e-05, 1.2119e-09,  ..., 1.2301e-08, 8.5260e-01,
         1.1514e-09]])

In [47]:
prob[0].sum() # normalized

tensor(1.0000)

In [48]:
Y # labels

tensor([ 1,  1,  2,  ..., 26, 24,  0])

In [49]:
torch.arange(36)

tensor([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
        18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35])

In [50]:
# prob[torch.arange(36), Y] # probabilities of the true characters

In [51]:
# loss = -prob[torch.arange(36), Y].log().mean() # cross entropy loss
# loss

In [52]:
# ----------------- Summary -----------------

In [53]:
X.shape, Y.shape

(torch.Size([212725, 3]), torch.Size([212725]))

In [74]:
g = torch.Generator().manual_seed(2147483647) # seed for reproducibility
C = torch.randn(27, 2, generator=g)
W1 = torch.randn((6, 100), generator=g)
b1 = torch.randn(100, generator=g)
W2 = torch.randn(100, 27, generator=g)
b2 = torch.randn(27, generator=g)
parameters = [C, W1, b1, W2, b2]

In [75]:
sum(p.nelement() for p in parameters) # total number of parameters

3481

In [76]:
# Enabling gradient computation
for p in parameters:
  p.requires_grad = True

In [77]:
# finding out the learning rate
lre = torch.linspace(-3, 0, 1000) # 1000 points between 0.001 and 1
lrs = 10 ** lre # learning rates
# lrs

In [84]:
lri = []
lossi = []

for i in range(10000): # iterations

# mini-batch construction
  ix = torch.randint(0, X.shape[0], (36,)) # 36 random indices from the dataset
# Forward pass

  emb = C[X[ix]] # embedding 
  h = torch.tanh(emb.view(-1, 6) @ W1 + b1) # hidden layer
  logits = h @ W2 + b2 # output layer
# counts = logits.exp() # unnormalized probabilities
# prob = counts / counts.sum(1, keepdim=True) # normalized probabilities
# loss = -prob[torch.arange(36), Y].log().mean() # cross entropy loss
#  but never use the above code in practice as it is not efficient and numerically unstable
  loss = F.cross_entropy(logits, Y[ix]) # cross entropy loss
  # print(loss.item())


  # Backward pass
  for p in parameters:
    p.grad = None

  loss.backward()

  # Update
  # lr = lrs[i]
  lr = 0.01
  for p in parameters:
    p.data += -lr * p.grad # learning rate e.g. -1 (lr)

  # track stats
  # lri.append(lre[i])
  # lossi.append(loss.item())

  
print(loss.item())    

1.9764630794525146


In [85]:
# plt.plot(lri, lossi)
# plt.xscale('log')

In [86]:
# F.cross_entropy(logits, Y) # PyTorch's implementation of cross entropy we can put the cross entropy directly in this code with the help of Pytorch directly in this code which will be more efficient for forward pass and backward pass and numerical stability.

In [87]:
logits.max(1) # max in python is a function that returns two values: the maximum value and its index

torch.return_types.max(
values=tensor([1.9932, 1.8313, 2.1193, 4.2734, 5.0452, 1.8055, 2.4242, 6.1538, 2.2210,
        2.7386, 1.8523, 6.1538, 5.0452, 1.9077, 1.8775, 1.0154, 2.5841, 6.7204,
        2.3665, 3.7490, 5.0452, 1.8627, 3.8568, 2.1801, 2.2299, 2.1812, 2.3324,
        5.0452, 2.3134, 6.7057, 2.0692, 3.4320, 5.7162, 2.5311, 5.0452, 2.9153],
       grad_fn=<MaxBackward0>),
indices=tensor([ 1,  0,  0, 12,  1, 12,  1,  1,  0,  0,  0,  1,  1,  1,  0,  0, 18,  1,
        18,  0,  1,  0,  5,  9,  0,  8, 14,  1,  0,  1,  0,  9,  1,  9,  1,  8]))

In [88]:
Y # labels

tensor([ 1,  1,  2,  ..., 26, 24,  0])

In [89]:
# creating minibatches
torch.randint(0, X.shape[0], (36,)) # random integers between 0 and 4

tensor([127386,  65398,  21875,  52761,  32588, 201112, 101033, 139296,  14254,
         60982, 173408,  29519, 177662, 156800,   9374, 208488,  65523, 164020,
        104050,  56987,  89606, 120780, 180868, 200198, 184173,  48206,   8006,
        116779, 101302,   9144, 125359, 208882, 140317,  94129,  71441,  66868])

In [92]:
# evaluation

emb = C[X] # embedding 
h = torch.tanh(emb.view(-1, 6) @ W1 + b1) # hidden layer
logits = h @ W2 + b2 # output layer
loss = F.cross_entropy(logits, Y) # cross entropy loss
loss


tensor(2.3433, grad_fn=<NllLossBackward0>)

In [ ]:
# Training split, dev / validation split, test split
# Training split: 80% of the data
# Dev split: 10% of the data
# Test split: 10% of the data